In [11]:
#-----------------------------------------------------------------------------------------------------------------------
# Reading data from sqlite
#-----------------------------------------------------------------------------------------------------------------------
import sqlite3
import pandas as pd

# #Establishing connection with sqlite database
conn = sqlite3.connect('../Sensors.db')
curs = conn.cursor()

df = pd.read_sql_query("SELECT * FROM BME_DATA", conn)
df.drop('id', axis=1, inplace=True)
df.rename(columns={'TIME_STAMP': 'TimeStamp',
                   'TEMPERATURE': 'Temperature',
                   'GAS':'Gas',
                   'HUMIDITY':'Humidity',
                   'PRESSURE':'Pressure',
                   'ALTITUDE':'Altitude'
                   }, inplace=True)
df.head()

df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%Y-%m-%d %H:%M:%S.%f')
df = df.resample('H', on='TimeStamp').agg({'Temperature':'mean', 'Gas':'mean','Humidity':'mean','Pressure':'mean','Altitude':'mean'})
df.reset_index(inplace = True)
df['Temperature'] = (df['Temperature'] * 9/5) + 32
df = df.round({'Temperature': 2, 'Gas': 2, 'Humidity': 2, 'Pressure':2, 'Altitude': 2})
df.head()
df.to_csv('BackUp.csv')

In [15]:
#-----------------------------------------------------------------------------------------------------------------------
# Pushing data into Azure mysql
#-----------------------------------------------------------------------------------------------------------------------
# Importing Dependencies
import mysql.connector
import pandas as pd
import json
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table
from mysql.connector import Error

df = pd.read_csv('../data/BackUp.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

with open("../paramteres/config.json") as config:
    param = json.load(config)

# Connecting to data warehouse
try:
    engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                      format(param['MyDemoServer'][0]['user'],
                                             param['MyDemoServer'][0]['password'],
                                             param['MyDemoServer'][0]['host'],
                                             param['MyDemoServer'][0]['database']), echo=False)

    # Cleaning the data from existing tables MetricValues and Metrics
    Epi_con = engine.connect()
    if Epi_con.connect():
        try:
            #Epi_con.execute("""TRUNCATE TABLE Sessions""")
            df.to_sql('sensordata', con=Epi_con, if_exists='append',chunksize=1000, index=False)
            Epi_con.close()
            engine.dispose()
        except OSError as e:
            print(e)
except OSError as e:
    print(e)

,TimeStamp,Temperature,Gas,Humidity,Pressure,Altitude
0,2021-01-12 19:00:00,69.21,355355.55,40.13,984.80,240.38
1,2021-01-12 20:00:00,67.62,541956.52,40.84,984.77,239.90
2,2021-01-12 21:00:00,67.18,579178.53,40.63,984.75,239.96
3,2021-01-12 22:00:00,66.60,577062.79,40.57,984.82,239.43
4,2021-01-12 23:00:00,66.22,578171.79,39.79,984.44,242.66
...,...,...,...,...,...,...
495,2021-02-02 10:00:00,68.62,710860.80,30.13,976.68,309.01
496,2021-02-02 11:00:00,69.88,695186.09,29.82,976.90,307.09
497,2021-02-02 12:00:00,69.09,688382.94,30.51,976.62,309.53
498,2021-02-02 13:00:00,67.16,733037.09,31.00,976.23,312.88


In [1]:
from datetime import datetime
import requests
import json

now = datetime.strftime(
    datetime.now(),
    "%Y-%m-%dT%H:%M:%S")
now

'2021-10-03T16:39:13'

In [ ]:
from datetime import datetime, timedelta
import time

while 1:

    print(datetime.now())
    dt = datetime.now() + timedelta(minutes=  1)
    dt = dt.replace(second = 1)

    while datetime.now() < dt:
        time.sleep(1)

2021-10-03 20:10:23.008216
2021-10-03 20:11:01.209158
2021-10-03 20:12:01.588089
2021-10-03 20:13:02.077171
2021-10-03 20:14:01.432136
2021-10-03 20:15:01.813256
2021-10-03 20:16:02.223510
2021-10-03 20:17:01.601347
2021-10-03 20:18:02.090097
2021-10-03 20:19:01.451514
2021-10-03 20:20:01.719511
2021-10-03 20:21:02.083852
2021-10-03 20:22:01.477463
2021-10-03 20:23:01.846351
2021-10-03 20:24:02.292763
2021-10-03 20:25:01.724617
2021-10-03 20:26:02.065107
2021-10-03 20:27:01.375988
2021-10-03 20:28:01.736766
2021-10-03 20:29:02.229352
2021-10-03 20:30:01.661640
2021-10-03 20:31:02.081628
2021-10-03 20:32:01.428350
2021-10-03 20:33:01.890654
2021-10-03 20:34:02.286908
2021-10-03 20:35:01.632521
2021-10-03 20:36:02.040000
2021-10-03 20:37:01.369274
2021-10-03 20:38:01.776436
2021-10-03 20:39:02.144128
2021-10-03 20:40:01.528702
2021-10-03 20:41:01.883687
2021-10-03 20:42:02.218066
2021-10-03 20:43:01.610973
2021-10-03 20:44:01.904264
2021-10-03 20:45:02.247592
2021-10-03 20:46:01.666161
2

datetime.datetime(2021, 10, 3, 21, 1, 49, 656284)